## Get the data
The below gets the data from the latest week.

In [118]:
# This is a method of scraping table data from online (generally tagged in html as <table>)
# Install packages
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import csv
import os

# Site URL
url="https://projects.fivethirtyeight.com/soccer-predictions/premier-league/"

# GET request to fetch the raw html content
html_content = requests.get(url).text

# Parse HTML code for the entire site
soup = BeautifulSoup(html_content, 'lxml')
#print(soup.prettify())

# Just checking how many tables there are on the page so we only get the one
premleague = soup.find_all("table")
print("Number of tables on site: ", len(premleague))

# For reproducibility, lets pretend there are more than one tables and we want just the first (the 0th index in Python speech)
# Scrape the first table
table1 = premleague[0]
# Head will form our column names 
body = table1.find_all("tr")
# Head values are teh first itesm of the body list
head = body[2] # ie the 0th item is the header row #note we can change this to 2 to get the right headings
bodyrows = body[3:] # every other item apart from the 0th make up the rest of the rows in the table # note we've changed this to 3rd item and beyond because 2nd item is the column

# Unsure if this will work but lets see
# Iterate through the head html code and make list of clean headings
headings = []
for item in head.find_all("th"): # loop through all the th elements
    # conver the the elements to text and strip \n
    item = (item.text).rstrip("\n")
    # append clean column name to headings
    headings.append(item)
print(headings)
# Issue is the table has two sets of headers - we need the second row.
# If we change head to body[2], we get the right values...

# Now to loop through the rest of the rows
all_rows = [] # this is going to be a list for list of all rows
for row_num in range(len(bodyrows)): # A row at a time
    row = [] # to hold old entires for one row
    for row_item in bodyrows[row_num].find_all("td"): # loop through all row entries
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma seperates thousands in numbers
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        # append aa to row - note one row entry is being appended
        row.append(aa)
    # append one row to all_rows
    all_rows.append(row)

# So now we can use the data on all_rowsa and heading to make a table
# all_rows becomes our data and headings the column names
df = pd.DataFrame(data=all_rows,columns=headings)
df.head()
print(df)
df

Number of tables on site:  1
['team', 'spi', 'off.', 'def.', 'goal diff.', 'proj. pts.pts.', 'Every position', 'relegatedrel.', 'qualify for UCLmake UCL', 'win Premier Leaguewin league']
                   team   spi off. def. goal diff. proj. pts.pts.  \
0       Man. City17 pts  93.0  3.0  0.3        +64             87   
1        Liverpool9 pts  89.0  2.8  0.5        +45             75   
2         Arsenal18 pts  81.5  2.3  0.6        +25             71   
3       Tottenham17 pts  81.7  2.4  0.6        +26             69   
4         Chelsea10 pts  81.3  2.2  0.5        +14             64   
5     Man. United12 pts  75.9  2.1  0.7         +7             61   
6        Brighton13 pts  76.3  2.0  0.6        +10             60   
7        Newcastle8 pts  72.9  1.9  0.7         +1             51   
8   Crystal Palace6 pts  70.2  1.8  0.8         -5             48   
9        Brentford9 pts  68.8  2.0  0.9         -4             48   
10     Aston Villa7 pts  71.0  1.8  0.7         -8    

,team,spi,off.,def.,goal diff.,proj. pts.pts.,Every position,relegatedrel.,qualify for UCLmake UCL,win Premier Leaguewin league
0,Man. City17 pts,93.0,3.0,0.3,+64,87,,<1%,96%,66%
1,Liverpool9 pts,89.0,2.8,0.5,+45,75,,<1%,76%,15%
2,Arsenal18 pts,81.5,2.3,0.6,+25,71,,<1%,59%,8%
3,Tottenham17 pts,81.7,2.4,0.6,+26,69,,<1%,55%,6%
4,Chelsea10 pts,81.3,2.2,0.5,+14,64,,<1%,35%,3%
5,Man. United12 pts,75.9,2.1,0.7,+7,61,,1%,25%,1%
6,Brighton13 pts,76.3,2.0,0.6,+10,60,,1%,24%,1%
7,Newcastle8 pts,72.9,1.9,0.7,+1,51,,6%,7%,<1%
8,Crystal Palace6 pts,70.2,1.8,0.8,-5,48,,10%,5%,<1%
9,Brentford9 pts,68.8,2.0,0.9,-4,48,,10%,4%,<1%


# Problems
There are two main problems:
    1. The team name and the points are in the same column. We want to seperate that (should be easy enough to seperate because the end of the team name is the first number of the points)
    2. We need each week and to join the data together.

# Solution to team name and points in same column

In [119]:
## There is probably a more elegant way to do this but this worked
# Add a space between the team name and number
df = df.assign(
    team=df["team"].str.extract(r'(\D+)') + " " + df["team"].str.extract(r'(\d+)')
)
# Extract the numbers out of the teams column and create a new column called points
df['points'] = df['team'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
# Delete remaining numbers in the team column
df['team'] = df['team'].str.replace('\d+', '')
# And get rid of the trailing white space at the end
df['team'] = df['team'].str.strip( )


/var/folders/zq/7wx_w44j3mx9t6gygp9d79000000gn/T/ipykernel_63126/703438220.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['team'] = df['team'].str.replace('\d+', '')
